In [1]:
import keras_bitcoin
import dataset
import collections
import pandas as pd
from itertools import permutations, combinations
from sklearn.model_selection import train_test_split
import moment

Using TensorFlow backend.


In [122]:
#df = pd.read_csv("../Data/LDA_Data/reduceData3.csv")
#df = dataset.put_on_label_dataset(df, "day", binary=False, values = [-0.5, -0.2, 0.2, 0.5])

df = dataset.get_labeled_dataset(number_of_file=100, binary= False, values = [-5, -0.2, 0.2, 5], from_date = "2018-01-01")
print(collections.Counter(df["label"]))

#df = dataset.put_on_label_dataset(df, "date")
df.head()
print(df.shape)
print(df.head())

Number of files loaded :  102
Counter({4: 8110, 1: 6881, 2: 6798, 5: 4566, 3: 1686})
(28041, 8)
                                                text  label        date  \
0          So how does that make it not a currency?       3  2018-01-03   
1  I didn't say it wasn't a currency, I'm just sa...      3  2018-01-03   
2  I don't think spam has ever been a problem in ...      3  2018-01-03   
3  It was electrum wallet bro ._.\r\nMy transacti...      3  2018-01-03   
4  My laptop has 4 GB Ram, I think it can handle ...      3  2018-01-03   

   score  nb_replies  stickied  label_m1  label_m2  
0      2           1     False         4         5  
1      1           0     False         4         5  
2      1           1     False         4         5  
3      1           0     False         4         5  
4      1           0     False         4         5  


In [38]:
texts = df["text"]
texts = keras_bitcoin.encode_data(texts)
df = df.drop(columns=["text"])
texts = list(texts[0])

In [28]:
sentence_length = len(texts[0])
for i in range(sentence_length):
    words = [texts[k][i] for k in range(len(texts))]
    df[f"word_{i+1}"] = words

In [123]:
df["year"] = [int(str(date)[0:4]) for date in df["date"]]
df["month"] = [int(str(date)[5:7]) for date in df["date"]]
df["day"] = [int(str(date)[8:10]) for date in df["date"]]
df = df.drop(columns=["date", "text", "nb_replies", "score", "stickied"])

In [124]:
texts_train, texts_test , labels_train, labels_test = train_test_split(df.drop(columns=["label"]), df["label"] , test_size = 0.20)

In [97]:
texts_train.head()

,label_m1,label_m2,year,month,day
4174,2,1,2018,1,9
9512,3,2,2018,1,18
17025,5,2,2018,2,8
23345,3,2,2018,3,13
9879,3,2,2018,1,18


In [22]:
texts = df["text"]
labels = df["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size\
= keras_bitcoin.get_train_test_data(texts, labels)

In [125]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xg_class = XGBClassifier(objective ='reg:tweedie',
                         max_depth = 4,
                         n_estimators = 200,
                         eta = 0.5,
                         )
xg_class.fit(texts_train,labels_train)
preds = xg_class.predict(texts_test)
accuracy = accuracy_score(labels_test, preds)
print(accuracy)

1.0


In [107]:
from sklearn.metrics import mean_squared_error
mean_squared_error(labels_test, preds)

0.0

In [100]:
df[-1:]


,label,label_m1,label_m2,year,month,day
28040,2,5,4,2018,4,12


In [126]:
df_t = dataset.get_labeled_dataset(number_of_file=100, binary= False, values = [-5, -0.2, 0.2, 5], from_date = "2018-04-12")
df_t["year"] = [int(str(date)[0:4]) for date in df_t["date"]]
df_t["month"] = [int(str(date)[5:7]) for date in df_t["date"]]
df_t["day"] = [int(str(date)[8:10]) for date in df_t["date"]]
df_t = df_t.drop(columns=["date", "text", "nb_replies", "score", "stickied"])


Number of files loaded :  102


In [127]:
l = df_t["label"]
df_t = df_t.drop(columns=["label"])

pred = xg_class.predict(df_t)
accuracy = accuracy_score(l, pred)
mse = mean_squared_error(l, pred)
print("acc :",accuracy, "mse : ", mse )

acc : 0.38135186960690315 mse :  2.2027804410354745


In [118]:
l = df_t["label"]
df_t = df_t.drop(columns=["label"])

pred = xg_class.predict(df_t)
accuracy = accuracy_score(l, pred)
mse = mean_squared_error(l, pred)
print("acc :",accuracy, "mse : ", mse )

acc : 0.32981783317353786 mse :  1.5387104506232023


In [121]:
l = df_t["label"]
df_t = df_t.drop(columns=["label"])

preds = xg_class.predict(df_t)
accuracy = accuracy_score(l, preds)
mse = mean_squared_error(l, preds)
print("acc :",accuracy, "mse : ", mse )

acc : 0.32550335570469796 mse :  1.5290028763183126


In [18]:
import pickle
pickle.dump(xg_class, open("xgboost_bitcoin.dat", "wb"))

In [20]:
loaded_model = pickle.load(open("xgboost_bitcoin.dat", "rb"))

preds = loaded_model.predict(texts_test)
accuracy = accuracy_score(labels_test, preds)
print(accuracy)

0.33145604395604394


In [24]:
res = {"0" : 0, "1" : 0, "2" : 0, "3" : 0, "4" : 0}
for i in range(len(labels_test)):
    val = abs(list(labels_test)[i] - list(preds)[i])
    res[str(val)] += 1


In [25]:
for k in res.keys():
    res[k] /= len(preds) 
res

{'0': 0.32362637362637364,
 '1': 0.20851648351648353,
 '2': 0.25535714285714284,
 '3': 0.20302197802197802,
 '4': 0.009478021978021977}

In [276]:
from sklearn.metrics import mean_squared_error
import random

mean_squared_error(labels_test, [random.randint(1, 5) for x in labels_test])

3.9578296703296703

In [285]:
epochs=5
batch_size=None
perfs = {}
dropouts = []


#reducedata2
#function_list = ['tanh', 'softmax', 'softsign', 'hard_sigmoid']
#function_list =  list(combinations(function_list, 3))
#function_list = ['softmax', 'softplus', 'sigmoid', 'hard_sigmoid', 'exponential']
#function_list =  list(combinations(function_list, 2))



#stem only
function_list = [['softsign', 'relu', 'sigmoid', "tanh"]]
#no dropout
#function_list = [['tanh', 'softmax'], ['tanh', 'softplus'], ['tanh', 'sigmoid'], ['tanh', 'hard_sigmoid'], ['tanh', 'exponential'], ['softmax', 'sigmoid'], ['softmax', 'hard_sigmoid'], 'softmax', 'exponential'], ['softplus', 'softsign'], ['softplus', 'sigmoid'], ['softplus', 'hard_sigmoid'], ['softplus', 'exponential'], ['softsign', 'sigmoid'], ['softsign', 'hard_sigmoid'], ['softsign', 'exponential'], ['sigmoid', 'hard_sigmoid'], ['hard_sigmoid', 'exponential']]
#function_list =  list(combinations(function_list, 2))
temp = []


#function_list =  list(combinations(["softmax", "softplus", "softsign", "sigmoid", "hard_sigmoid"], ))
#scrambled_list = [ 'tanh', 'softmax' ,'softplus', 'sigmoid', 'hard_sigmoid', 'exponential','linear']
#random.shuffle(scrambled_list)
#function_list =  list(combinations(scrambled_list, 3))
#function_list =  list(combinations(keras_bitcoin.available_activation_functions, 3))

size = len(function_list)
count = 0
for function in function_list:
    count += 1
    print(count, size)
    if isinstance(function, str):
        function = [function]
    else :
        function = list(function)
        dropouts = [0.1, 0.1]
    model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length,\
                                    max_sentence_size, epochs=epochs,\
                                    batch_size=batch_size, activations_functions=function,\
                                    dropouts=dropouts, verbose = 0)
    loss, accuracy = model.evaluate(texts_test, labels_test, verbose=0)
    preds = model.predict(texts_test)
    if accuracy >= 0.5:
        temp.append(function)
    perfs["_".join(function)] = [accuracy, mean_squared_error(labels_test, preds)]

1 1


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [288]:
perfs

{'softsign_relu_sigmoid_tanh': [0.0, 15.333236874968053]}

In [19]:
df_test = dataset.get_labeled_dataset(number_of_file = 100, from_date = "2017-02-10", date_included = False)

Number of files loaded :  10


In [20]:
preds = keras_bitcoin.get_predictions(list(df_test["text"]), model, vocab_length, max_sentence_size)
df_test["preds"] = list(map(lambda x : int(x), preds))
df_test["correct"] = np.equal(preds, df_test["label"])

In [21]:
dataset.get_prediction_stats(df_test)

Number Correct/Wrong Guess : 5/363
              Accuracy : 1.358695652173913
Invalid sentences count 0


In [ ]:
scrambled_list = list(random.shuffle([ 'tanh', 'softmax' ,'softplus', 'sigmoid', 'hard_sigmoid', 'exponential','linear']))


In [14]:
from sklearn.metrics import confusion_matrix
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(y_true, y_pred)

array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]], dtype=int64)